<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src='OCR_logo3.png'>

# Selecting sensors for a stress detection wearable device

_Version 1.0_  
_Author(s): Jon Reifschneider, Duke University Pratt School of Engineering_

<img align="left" style="padding-top:10px;" src="Respiban.png">  

## _About this teaching case_
**Level:** Beginner  
**Language:** Python  
**Libraries:** Pandas, Matplotlib  
**Industry:** Healthcare

**Learning Topics:**  
- Data Manipulation
- Exploratory Data Analysis
- Outliers  

**Learning Objectives**   
- Learn to read in, clean and filter raw sensor data
- Learn strategies for identifying and removing outliers in data
- Build skills in exploratory data analysis using visual and statistical analysis techniques

**Pre-requisites**  
- Basic proficiency in Python and pandas

**Case Structure**  
This teaching case is structured to follow the ***Modified CRISP-DM Data Science Process*** used in Duke University's AI for Product Innovation graduate programs. 

**Datasets Used**  
Philip Schmidt, Attila Reiss, Robert Duerichen, Claus Marberger and Kristof Van Laerhoven. 2018. Introducing WESAD, a multimodal dataset for Wearable Stress and Affect Detection. In 2018 International Conference on Multimodal Interaction (ICMI ’18), October 16–20, 2018, Boulder, CO, USA. ACM, New York, NY, USA, 9 pages. https://doi.org/10.1145/3242969.3242985

Originial (unadjusted) dataset can be found at https://ubicomp.eti.uni-siegen.de/home/datasets/icmi18/. The dataset used in this exercise has been adapted from the original to facilitate the learning objectives of this exercise.

# Contents
[1: Business Understanding](#1)  
[2: Data Understanding](#2)  
&nbsp;&nbsp;[2.1: Gather Data](#3)  
&nbsp;&nbsp;[2.2: Validate Data](#4)   
&nbsp;&nbsp;[2.3: Explore Data](#5) 

In [ ]:
# Run this before any other code cell
# This downloads the csv data files into the same directory where you have saved this notebook

import urllib.request
from pathlib import Path
import os
path = Path()

# Dictionary of file names and download links
files = {'S2_respiban_adjusted.txt':'https://storage.googleapis.com/aipi_datasets/S2_respiban_adjusted.txt'}

# Download each file
for key,value in files.items():
    filename = path/key
    url = value
    # If the file does not already exist in the directory, download it
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url,filename)

# Step 1: Business Understanding <a class="anchor" id="1"></a>

## 1.1 Define the problem
Long-term stress has been proven to have harmful health effects, ranging from headaches, sleeping difficulties, and even increased risk for cardiovascular diseases.  The ability for individuals to automatically detect signs of ongoing stress can help them to seek intervention to reduce stress levels, resulting in a reduction in harmful side-effects. 

You are an engineer working for a company developing a wearable chest-strap stress detection device which uses sensors to detect physiological signs of stress and then alerts the user so they can take proactive action to reduce stress, in order to mitigate potential long-term health effects.  You have been asked to lead a project to determine which sensors need to be included on the new wearable device in order to accurately detect stress.

## 1.2 Define success
The expected impact of the company's product will be to enable early mitigation of ongoing stress for users, resulting in a reduction in negative health impacts due to stress, e.g. headaches. In order to achieve this, the company need to be able to accurately identify periods of stress as compared to non-stress periods. 

## 1.3 Identify factors

From your interviews of domain experts, you learn that stress is a physiological response to stimulus triggered by the sympathetic nervous system.  During this response, hormones are released that lead to physiological changes such as increased respiratory rate, increased heart rate, muscle tension, and increased motion.  In order to determine which sensors may be needed to provide input into the model to predict stress, you have developed a list of physiological and motion signals which can be feasibly measured by sensors on a non-invasive wearable device:  
- Motion/acceleration
- Electrocardiography
- Electrodermal activity
- Electromyography
- Respiration
- Skin temperature

Your plan is to evaluate each of these signals to identify which ones contain patterns that may provide an indicator of stress.  This will help you narrow down the list of sensors needed on the wearable device to only those which have value for collecting data to drive a stress identification model.

# Step 2: Data Understanding <a class="anchor" id="2"></a>

You have gathered data you plan to use for an initial analysis from the Wearable Stress and Affect Detection (WESAD) study done by engineers at Bosch which includes high-resolution data from several sensors together with labels of the subject's mood at each point in time over the duration of the study period.  The sensors are contained on two devices - one chest-based and the other wrist-based.  The description of the data in the study is below.  The study contains data on several participants - for the initial analysis you will use the data for just one individual (S2) from the RespiBAN chest sensor array.

**Description of the dataset:**  
The study authors used the RespiBAN chest sensor to measure physiological signals and motion: http://www.biosignalsplux.com/en/respiban-professional. All signals were sampled at 700 Hz. Raw data is contained in SX_respiban.txt. There are 10 columns here. First column: sequential line number. Second column: ignore. Columns 3-10: raw data of the 8 sensor channels. The order of the channels is defined in the header. The entries “XYZ” refer to the 3-channel accelerometer (thus, acceleration data is provided in 3 columns). In order to convert the raw sensor values into SI units, each channel has to transformed based on the formulas given below (signal contains the raw sensor values, vcc=3, chan_bit=2^16).
- ECG (mV): ((signal/chan_bit-0.5)*vcc) Details: http://www.biosignalsplux.com/datasheets/ECG_Sensor_Datasheet.pdf
- EDA (μS): (((signal/chan_bit)*vcc)/0.12) Details: http://www.biosignalsplux.com/datasheets/EDA_Sensor_Datasheet.pdf
- EMG (mV): ((signal/chan_bit-0.5)*vcc) Details: http://www.biosignalsplux.com/datasheets/EMG_Sensor_Datasheet.pdf
- TEMP (°C):  
    vout = (signal*vcc)/(chan_bit)  
    rntc = ((10^4)*vout)/(vcc-vout)  
    TEMP = - 273.15 + 1./(1.12764514*10^(-3) + 2.34282709*10^(-4)*log(rntc) + 8.77303013*10^(-8)*(log(rntc))^3)  
    Details: http://www.biosignalsplux.com/datasheets/TMP_Sensor_Datasheet.pdf
- XYZ (g): (signal-Cmin)/(Cmax-Cmin)*2-1, where Cmin = 28000 and Cmax = 38000  
Details: http://www.biosignalsplux.com/datasheets/ACC_Sensor_Datasheet.pdf
- RESPIRATION (%): (signal / chan_bit - 0.5) * 100 Details: http://www.biosignalsplux.com/datasheets/PZT_Sensor_Datasheet.pdf

## 2.1 Gather data <a class="anchor" id="3"></a>

In [ ]:
# Run setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings("ignore")

# Read in the raw data from the txt file
raw_data = pd.read_csv('S2_respiban_adjusted.txt',sep = '\t',skiprows=3,header=None,)
chest_data = raw_data.copy() # Work with a copy to preserve the raw dataframe

# Read in the metadata
with open('S2_respiban_adjusted.txt') as f:
    lines = f.readlines()
metadata = lines[1]

# Read in the labels csv to a dictionary containing label and start and end time for each
labels = {}
with open('S2_labels.csv') as f:
    lines = f.readlines()
for line in lines[1:]:
    key = line.split(';')[0].strip('# ')
    values = [line.strip() for line in line.split(';')[1:]]
    labels[key] = values
# Convert start and end times to float
labels['START'] = [float(x) for x in labels['START']]
labels['END'] = [float(x) for x in labels['END']]

Use Python string operations to extract the column names (signals) from the metadata and put them into a list called *colnames*. The column names are contained in a list with the key "sensor" in the metadata dictionary.  They should be stored in the list as strings with no additional punctuation or white space around them. E .g. ['ECG', 'EDA', ...]

Delete columns 0,1,10 from the *chest_data* dataframe per the data description.  Then, rename the columns to the column names you just extracted from the metadata

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

# Check to confirm correct values in colnames
assert colnames == ['ECG', 'EDA', 'EMG', 'TEMP', 'XYZ1', 'XYZ2', 'XYZ3', 'RESPIRATION'], 'colnames is incorrect'

Let's add a time attribute to our data.  The chest sensor and label data is sampled at 700 Hz  

Convert the dataframe index to a timedelta index with the values in seconds.  The chest sensor and label data is sampled at 700 Hz

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

# Check that total time is correct
assert round(np.max(chest_data.index.total_seconds())/60,ndigits=0) == 106

The ground truth mood labels for the data are contained in S2_labels.csv, which has been read in to a dictionary *labels* containing label and start and end time for each.  Add a 'label' column to your dataframe containing the corresponding string label for each row, based on start/end time of each label

In [ ]:
### BEGIN SOLUTION

    
### END SOLUTION

## 2.2 Validate data  <a class="anchor" id="4"></a>
### Transform raw sensor data into proper units  
Use the transfer function equations given in the sensor spec sheets (and the description above in the Data Understanding section) to convert the raw values into the proper units.  Write a function *unitize(df,vcc,chan_bit,Cmax,Cmin)* to convert the raw data values from ALL sensors in the input dataframe to proper units.  As an example, the calculation for temperature conversion is provided.  The inputs are the raw data dataframe *df* and variables *vcc*, *chan_bit*, *Cmax* and *Cmin*.  The function returns a dataframe *df_units* with the same columns as the original input dataframe, converted into proper units.**

In [ ]:
def unitize(df, vcc, chan_bit, Cmax, Cmin):
    df_units = df.copy()
    
    ### BEGIN SOLUTION
  
   
    ## END SOLUTION
    
    return df_units

Use your new function to convert chest_data into proper units.  Let's also combine our three components of acceleration to get one acceleration factor with the total magnitude of acceleration.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

### Clean up missing and erroneous data

Check to see if we have missing data in any of our columns

In [ ]:
# Check to see if we have missing data in any of our columns
chest_data_units.isna().sum(axis=0)

As we can see above, we have several rows of data with null values for sensor readings.  What might be some possible reasons why we have missing data in this case?

Since we are dealing with a time series, rather than simply discarding the missing values an alternative approach would be to use a backwards or forward fill to fill them in using the values from the previous or the next observation from the sensors.  

Define a function clean_data() which takes as input a dataframe containing data to clean (*chest_data_units*) and a list of labels to filter on (*relevant_labels*).  The function should first filter the dataframe to only contain rows with a label included in *relevant_labels*.  It should then fill the null signal values in the filtered dataframe using a forward-fill approach (set them equal to the last observed value in time before the null value).  The function should return the cleaned and filtered dataframe as *df_clean*.  Once we have filtered and filled null signal values, we should have no null values left in the data

In [ ]:
relevant_labels=['Baseline','Stressed','Amusement']

def clean_data(df, relevant_labels):
    ### BEGIN SOLUTION
    
    
    ### END SOLUTION
    
    return df_clean

In [ ]:
chest_data_clean = clean_data(chest_data_units,relevant_labels)

# Check to see if all null values have been filled
assert chest_data_clean.isna().sum(axis=0).sum()==0, 'The dataframe still contains null values!'

### Check for anomalous values

**Visual check for outliers**  
A good place to start in looking for anomalous values is by visually evaluating the data. Let's do this by creating a simple plot of each signal over the period, and observe whether we see unusual values

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

From the above plots, it looks like there may be a few severe outliers in the data.  Select an appropriate approach to identifying extreme outliers in the data, and then remove the extreme outliers from your dataframe.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

## 2.3 Explore the data <a class="anchor" id="5"></a>

Our data is now in proper units, filtered for modeling, and scrubbed to remove null values and anomalies.  Now we are ready to explore the data to identify trends and patterns which we can use to determine which sensors help us most in identifying periods of stress.

For each signal, visually analyze the signal values for each mood label.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

Next, compute and compare the mean and standard deviation values of each group

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

For the signals which appear to have a large difference in the mean of "stressed" (label==2) relative to the other two mood states, use an ANOVA to determine if the difference between the mean signal values for the groups is statistically significant at a level of 0.05

In [ ]:
### BEGIN SOLUTION

    
### END SOLUTION

Finally, for any signals that you suspect provide a good indicator of stress, it is a good idea to plot the data again, this time identifying the set of values corresponding to each mood using a different color so that we can visually compare the trend in values for each mood.  Why do we do this? Because sometimes there are other indicators we might look for in the data beyond a difference in means between groups (e.g. one group might have a much higher variance than the others, or there might be an increasing/decreasing trend in values of one group not seen in the others

In [ ]:
### BEGIN SOLUTION


## END SOLUTION

Based on the visual and statistical analyses you ran above, which sensors do you recommend including on our wearable stress detector device? Why? Please justify your answer by referring to the data and plots